# Application example | **Embedding of protein families across protein language models**

This notebook demonstrates how to use the ema-tool library to analyse the embedding of protein families across protein language models. We will use the [ESM Model](https://github.com/facebookresearch/esm) model and a set of ion channel wildtype proteins as an example.

## Loading dependencies

In [ ]:
import numpy as np
import pandas as pd

from ema import EmbeddingHandler

## Loading data

`ema-tool` requires a table of **metadata** about the proteins to be embedded. The first column of the table should contain the sample identifiers. These need to be unique. Any further columns can contain metadata about the samples. In this example, we will use a table of ion channel proteins. Metadata can be either categorical or numerical.

Further the **embedding spaces** of the proteins need to be provided in the format of NumPy arrays. In this example, we will use the ESM-1b model to embed the proteins. Each row in the NumPy array should correspond to the embedding of the protein with the same row index in the metadata table. The dimensions of the embedding space can differ between models.

In [ ]:
DATA_DIR = '../examples/HCN1-variants/'
FP_METADATA = DATA_DIR + 'metadata.csv'
FP_EMB_ESM1b = DATA_DIR + 'esm1b_t33_650M_UR50S-embeddings.npy'
FP_EMB_ESM2 = DATA_DIR + 'esm2_t33_650M_UR50D-embeddings.npy'

In [ ]:
# load metadata and embeddings 

metadata = pd.read_csv(FP_METADATA)
emb_esm1b = np.load(FP_EMB_ESM1b)
emb_esm2 = np.load(FP_EMB_ESM2)

print(emb_esm1b.shape, emb_esm2.shape)
metadata.head()

## Initialising the ema object

The `EmbeddingHandler` object is the main object in the ema-tool library. It is used to load the model, preprocess the data, and perform the analysis.
It is initialised with the metadata of the analysed samples. 

Embedding spaces are added to the object using the `add_emb_space` method. Multiple embedding spaces can be added to the object. Each embedding space is identified by a unique name. Embedding spaces can also be removed from the object using the `remove_emb_space()` method.

In [ ]:
# initialize embedding handler
emb_handler = EmbeddingHandler(metadata)

# add embeddings to the handler
emb_handler.add_emb_space(embeddings=emb_esm1b, emb_space_name='esm1b')
emb_handler.add_emb_space(embeddings=emb_esm2, emb_space_name='esm2')

## Accessing the metadata

The sample names can be accessed using the `get_sample_names()` method.

In [ ]:
emb_handler.get_sample_names()[:5]

The columns of the metadata can be accessed shown using the `get_groups()` function. The function returns a list of the column names.

In [ ]:
emb_handler.get_groups()

The `get_unique_values()` function can be used to get the unique values of a specific column in the metadata.

In [ ]:
emb_handler.get_unique_values_per_group(
    group="alphafold2_ss9"
)

The `get_value_count_per_group` function can be used to get the number of samples per unique value in a specific column.

In [ ]:
emb_handler.get_value_count_per_group(
    group="alphafold2_ss9"
)

The `get_samples_per_group_value()` parameter allows to get the rows of the metadata table that correspond to a specific value in a specific column.

In [ ]:
emb_handler.get_samples_per_group_value(
    group="alphafold2_ss9",
    group_value="B"
)

## Analysis Within an Embedding Space

### Distribution of embedding values

Multiple analyses can be performed to understand the scale and distribution of each embedding space.

This histogram shown with the `plot_emb_hist()` displays the distribution of the embedding values stratified by the embedding space. For an embedding of dimension d=1280 all 1280 values of the embedding will be visualised independently.

In [ ]:
emb_handler.plot_emb_hist()

The `plot_emb_box()` function can be used to display the distribution of the embedding values in a boxplot. 

Here the `group` parameter can be used to stratify the samples by a specific column in the metadata or by individual samples.

In [ ]:
emb_handler.plot_emb_box(group="sample")

In [ ]:
emb_handler.plot_emb_box(group="data_source")

### Unsupervised clustering

During the initialisation of the `EmbeddingHandler` object, an unsupervised clustering can be performed automatically using the KMeans algorithm. 

By default ema computes a number of clusters equal to the mean number of categories in the metadata. This is a good starting point, but you can also specify the number of clusters you want to compute.


In [ ]:
# specify a number of unsupervised clusters

emb_handler.recalculate_clusters(
    n_clusters=5,
    emb_space_name='esm1b')

emb_handler.recalculate_clusters(
    n_clusters=5,
    emb_space_name='esm2'
    )    

The agreement between the clustering and the metadata can be visualised using the `plot_feature_cluster_overlap()` function. This function shows a barplot of the number of samples for each metadata value of the specified metadata column (selected with the `feature` parameter). The colour of the bars represents the cluster assignment of the samples. The height of the bars represents the number of samples in the dataset with the corresponding metadata value.

In [ ]:
emb_handler.plot_feature_cluster_overlap(
    emb_space_name = "esm1b",
    feature="alphafold2_ss3"
)

The function can also be used to compare the agreement between the unsupervised clustering of two different embedding spaces. By setting the `feature` parameter to `cluster_<emb-space-name>` the function will compare the agreement between the unsupervised clustering of the two embedding spaces.

In [ ]:
emb_handler.plot_feature_cluster_overlap(
    emb_space_name = "esm1b",
    feature="cluster_esm2"
)

### Visualisation of dimensionality reduction

Each embedding space can be visualised using dimensionality reduction techniques such as PCA, t-SNE and UMAP. The colour of the points can be stratified by a specific column in the metadata. If the `color` parameter is not set, the points will be coloured by the unsuperivsed clustering.

#### PCA

In [ ]:
emb_handler.visualise_emb_pca(emb_space_name="esm1b", 
                              colour="uniprot_region"
                              )

In [ ]:

emb_handler.visualise_emb_pca(emb_space_name="esm2", 
                              colour="uniprot_region")

#### t-SNE

In [ ]:
emb_handler.visualise_emb_tsne(emb_space_name="esm1b")

In [ ]:
emb_handler.visualise_emb_umap(
    emb_space_name="esm1b",
    colour="alphamissense_class_benign"
)

### Pairswise distances between samples within an embedding space

The following distance metrics are available for the comparison of pairwise distances between samples:

*   `cosine`: Cosine distance
*   `adjusted_cosine`: Adjusted cosine distance
*   `cityblock`: Manhattan distance
*   `cityblock_normalised`: Normalised manhattan distance
*   `euclidean`: Euclidean distance
*   `sequclidean`: Standardised euclidean distance
*   `sqeuclidean_normalised`: Standardised normalised euclidean distance
*   `euclidean_normalised`: Normalised euclidean distance
*   `mahalanobis`: Mahalanobis distance

The distances are automatically computed as soon as one of the distance functions is called.

All pairwise distances can be accessed in a heatmap. The x and y axes of the heatmap represent the samples. The colour of the heatmap represents the distance between the samples. The samples on each axis can be sorted based on a column in the metadata by using the `x_order` and `y_order` parameters.

In [ ]:
emb_handler.plot_emb_dis_heatmap(
    emb_space_name="esm1b",
    distance_metric="euclidean",
)

The matrix of pairwise distances can be accessed by using the `get_sample_distance()` function. The function returns a NumPy array of the pairwise distances.

In [ ]:
emb_handler.get_sample_distance(
    emb_space_name="esm1b",
    metric="euclidean"
)

The pairwise distances can be normalised with a normal distribution by setting the `rank` parameter to `normal_dis` for all functions that compute pairwise distances. The normalisation is performed by ranking the distances and transforming them to a normal distribution. To inspect whether the pairwise distances in an embedding space are normally distributed, the `plot_emb_dis_his_with_fitted_functions()` function can be used for a visual inspection.

In [ ]:
emb_handler.plot_emb_dis_his_with_fitted_functions(
    emb_space_name="esm1b",
    distance_metric="euclidean",
    rank="normal_dis"
)

### Correlation of pairwise distances with metadata

The correlation of pairwise distances with metadata can be computed using the `plot_emb_dis_continuous_correlation()` function. The function computes the correlation of the pairwise distances with a specific column in the metadata.

In [ ]:
emb_handler.plot_emb_dis_continuous_correlation(
    emb_space_name="esm1b",
    distance_metric="euclidean",
    feature="flDPnn_disorder_propensity"
)

The distances one space can be statified by a specific column in the metadata. The `group` parameter can be used to stratify the samples by a specific column in the metadata.

In [ ]:
emb_handler.plot_emb_dis_per_group(
    emb_space_name="esm1b",
    distance_metric="euclidean",
    group="alphafold2_ss3"
)

The plot can further be stratified by a specific value in the selected column by using the `group_value` parameter.

In [ ]:
emb_handler.plot_emb_dis_per_group(
    emb_space_name="esm1b",
    distance_metric="euclidean",
    group="alphafold2_ss3",
    group_value="H",
    plot_type="box"
)

## Comparing embedding spaces

The distribution of pairwise distances between samples in two embedding spaces can be compared using the `plot_emb_dis_hist` function. The function displays a histogram of the pairwise distances between samples in two embedding spaces.

In [ ]:
emb_handler.plot_emb_dis_hist(distance_metric = "euclidean")

The pairwise distances between two embedding spaces can be visualised using a scatter plot. The `plot_emb_dis_scatter()` function displays the pairwise distances between samples in two embedding spaces. The x and y axes of the scatter plot represent the pairwise distances between samples in the two embedding spaces. The `colour_group` and `colour_value_1` parameters define the column in the metadata and the value in the column which is used to stratify the samples by colour.

In [ ]:
emb_handler.plot_emb_dis_scatter(
    emb_space_name_1 = "esm1b",
    emb_space_name_2 = "esm2",
    distance_metric = "euclidean",
    colour_group = "alphafold2_ss3",
    colour_value_1 = "H",
)

In [ ]:
emb_handler.plot_emb_dis_scatter(
    emb_space_name_1 = "esm1b",
    emb_space_name_2 = "esm2",
    distance_metric = "euclidean",
    colour_group = "alphafold2_ss3",
    colour_value_1 = "H",
    colour_value_2 = "B",
)

The distances can be statified by a specific column in the metadata. The `group` parameter can be used to stratify the samples by a specific column in the metadata.

In [ ]:
emb_handler.plot_emb_dis_box(
    group = "alphafold2_ss3",
    distance_metric="euclidean",
)